In [1]:
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
import regex as re
from sklearn import svm

In [2]:
emb = None

def load_data():
    df1 = pd.read_csv("data_train_neg.csv")
    df2=pd.read_csv("data_train_pos.csv")
    frames=[df1,df2]
    resulst=pd.concat(frames)
    return resulst
# 2. Standardize and tokenizer Text column
def remove_icon(text):
    pattern = re.compile('[\W_]+')
    dirty_list = text.split()
    clean_list = [pattern.sub('', word) for word in dirty_list]
    clean_string = ' '.join(clean_list)
    return clean_string


def standardize_data(post):
    post=str(post)
    post=remove_icon(post)
    post=post.replace('.','')
    post=post.replace(',','')
    post=post.replace(' k ',' không ')
    post=post.replace(' mk ',' mình ')
    post=post.replace('\n',' ')
    post=post.replace(' cx ',' cũng ')
    post=post.replace('  ',' ')
    post=post.replace(' r ',' rồi ')
    post=post.replace(' dc ',' được ')
    post=post.strip()
    post = re.sub(r"[\.,\?]+$-", "", post)
    return post

def tokenizer(row):
    row=str(row)
    return word_tokenize(row, format="text")

# Embeding
def embedding(X_train, X_test):
    global  emb
    emb = TfidfVectorizer(min_df=5, max_df=0.8,max_features=3000,sublinear_tf=True)
    emb.fit(X_train)
    X_train =  emb.transform(X_train)
    X_test = emb.transform(X_test)

    # Save pkl file
    joblib.dump(emb, 'tfidf.pkl')
    return X_train, X_test



In [3]:
#  Load data from file
data = load_data()
data["Text"] = data.Text.apply(standardize_data)
data["Text"] = data.Text.apply(tokenizer)

In [4]:
#  Convert to X_train, y_train
X_train,X_test,y_train,y_test = train_test_split(data["Text"],data["Sentiment"],test_size=0.2, random_state=42)

# Embeding X_train
X_train,X_test  = embedding(X_train, X_test)

# Train and save model
model = svm.SVC(kernel='linear', C = 1)
model.fit(X_train,y_train)
joblib.dump(model, 'saved_model.pkl')

# Test
print("Model score=", model.score(X_test, y_test))
print("Done")

Model score= 0.8877263581488933
Done
